In [ ]:
import sqlite3
import urllib.request
import pandas as pd

url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
urllib.request.urlretrieve(url, "Chinook.sqlite")

conn = sqlite3.connect("Chinook.sqlite")
cursor = conn.cursor()

In [18]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in Chinook database:")
pd.DataFrame(tables, columns=['Customer'])

Tables in Chinook database:


,Customer
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [23]:
# 2. Try some queries to verify that the tables and relations are working.
print("\n--- Verifying Queries ---")


query1 = "SELECT * FROM Artist LIMIT 5;"
df1 = pd.read_sql_query(query1, conn)
print("\nFirst 5 Artists:")
print(df1)


query2 = "SELECT * FROM Track LIMIT 5;"
df2 = pd.read_sql_query(query2, conn)
print("\nFirst 5 Tracks:")
print(df2)


query3 = "SELECT * FROM Customer WHERE Country = 'USA' LIMIT 5;"
df3 = pd.read_sql_query(query3, conn)
print("\nFirst 5 Customers from USA:")
print(df3)



--- Verifying Queries ---

First 5 Artists:
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains

First 5 Tracks:
   TrackId                                     Name  AlbumId  MediaTypeId  \
0        1  For Those About To Rock (We Salute You)        1            1   
1        2                        Balls to the Wall        2            2   
2        3                          Fast As a Shark        3            2   
3        4                        Restless and Wild        3            2   
4        5                     Princess of the Dawn        3            2   

   GenreId                                           Composer  Milliseconds  \
0        1          Angus Young, Malcolm Young, Brian Johnson        343719   
1        1                                               None        342562   
2        1  F. Baltes, S. Kaufman, U. Dirkscneider &

In [20]:
# 3. Try a query statement that joins across at least three tables.
print("\n--- Join Across Three Tables ---")

query4 = """
SELECT c.FirstName, c.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId
JOIN Track t ON ii.TrackId = t.TrackId
LIMIT 10;
"""
df4 = pd.read_sql_query(query4, conn)
print("\nCustomer, Invoice Date, and Track Name:")
print(df4)


--- Join Across Three Tables ---

Customer, Invoice Date, and Track Name:
  FirstName LastName          InvoiceDate              TrackName
0    Leonie   Köhler  2009-01-01 00:00:00      Balls to the Wall
1    Leonie   Köhler  2009-01-01 00:00:00      Restless and Wild
2     Bjørn   Hansen  2009-01-02 00:00:00  Put The Finger On You
3     Bjørn   Hansen  2009-01-02 00:00:00       Inject The Venom
4     Bjørn   Hansen  2009-01-02 00:00:00             Evil Walks
5     Bjørn   Hansen  2009-01-02 00:00:00     Breaking The Rules
6      Daan  Peeters  2009-01-03 00:00:00            Dog Eat Dog
7      Daan  Peeters  2009-01-03 00:00:00               Overdose
8      Daan  Peeters  2009-01-03 00:00:00    Love In An Elevator
9      Daan  Peeters  2009-01-03 00:00:00      Janie's Got A Gun


In [ ]:
print("\n--- Timing Unindexed Join Query (10 Runs) ---")

# First, drop indexes that would normally optimize these joins
cursor.executescript("""
DROP INDEX IF EXISTS IFK_CustomerInvoice;
DROP INDEX IF EXISTS IFK_InvoiceLineInvoiceId;
DROP INDEX IF EXISTS IFK_InvoiceLineTrackId;
""")
conn.commit()

# Define our unindexed join query (same as query5 but now without index benefits)
unindexed_query = """
SELECT c.FirstName, c.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId
JOIN Track t ON ii.TrackId = t.TrackId
LIMIT 10;
"""

# Time the unindexed query
times_unindexed = []
for _ in range(10):
    # Vary the limit to avoid cache effects
    current_limit = random.randint(5, 15)
    varied_query = unindexed_query.replace('LIMIT 10', f'LIMIT {current_limit}')
    
    start_time = time.time()
    df_unindexed = pd.read_sql_query(varied_query, conn)
    end_time = time.time()
    times_unindexed.append(end_time - start_time)


cursor.executescript("""
CREATE INDEX IF NOT EXISTS IFK_CustomerInvoice ON Invoice(CustomerId);
CREATE INDEX IF NOT EXISTS IFK_InvoiceLineInvoiceId ON InvoiceLine(InvoiceId);
CREATE INDEX IF NOT EXISTS IFK_InvoiceLineTrackId ON InvoiceLine(TrackId);
""")
conn.commit()


print(f"Unindexed join average time: {sum(times_unindexed)/len(times_unindexed):.4f} seconds")
print(f"Individual times: {[round(t, 4) for t in times_unindexed]}")


--- Timing Unindexed Join Query (10 Runs) ---
Unindexed join average time: 0.0006 seconds
Individual times: [0.0012, 0.0007, 0.0005, 0.0006, 0.0004, 0.0005, 0.0005, 0.0005, 0.0004, 0.0005]


In [25]:
# 5. Find a way to time the query from #4 using tools available. Find the average of 10 queries. Vary the details a bit to avoid cache effects.
print("\n--- Timing Unindexed Query (10 Runs) ---")
import time
import random
times = []

query5 = """
SELECT c.FirstName, c.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId
JOIN Track t ON ii.TrackId = t.TrackId
LIMIT 10;
"""
for _ in range(10):
    start_time = time.time()
    df5 = pd.read_sql_query(query5, conn)
    end_time = time.time()
    times.append(end_time - start_time)
    # Vary the limit to avoid cache effects
    query5_vary = query5.replace('LIMIT 10', f'LIMIT {random.randint(5, 15)}')

print(f"Average time: {sum(times) / len(times)} seconds")



--- Timing Unindexed Query (10 Runs) ---
Average time: 0.0008382558822631836 seconds


In [ ]:
# 6. Create some index capabilities for the tables that will speed up the queries.

print("\n--- Creating Indexes ---")

cursor.executescript("""
CREATE INDEX IF NOT EXISTS idx_CustomerId ON Customer(CustomerId);
CREATE INDEX IF NOT EXISTS idx_InvoiceId ON Invoice(InvoiceId);
CREATE INDEX IF NOT EXISTS idx_TrackId ON Track(TrackId);
CREATE INDEX IF NOT EXISTS idx_InvoiceLine_InvoiceId ON InvoiceLine(InvoiceId);
CREATE INDEX IF NOT EXISTS idx_InvoiceLine_TrackId ON InvoiceLine(TrackId);

-- Additional indexes that might help with our specific query
CREATE INDEX IF NOT EXISTS idx_Customer_Country ON Customer(Country);
CREATE INDEX IF NOT EXISTS idx_Invoice_CustomerId ON Invoice(CustomerId);
""")
conn.commit()




--- Creating Indexes ---


In [31]:
# 7. Time the queries again, to see if the effect worked.
print("\n--- Timing Indexed Query (10 Runs) ---")

query6 = """
SELECT c.FirstName, c.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId
JOIN Track t ON ii.TrackId = t.TrackId
LIMIT 10;
"""

times_indexed = []
for _ in range(10):
    start_time = time.time()
    df6 = pd.read_sql_query(query6, conn)
    end_time = time.time()
    times_indexed.append(end_time - start_time)
    query6_vary = query6.replace('LIMIT 10', f'LIMIT {random.randint(5, 15)}')

print(f"Average time (indexed): {sum(times_indexed) / len(times_indexed)} seconds")


--- Timing Indexed Query (10 Runs) ---
Average time (indexed): 0.0006811857223510742 seconds


In [28]:
# 8. Look at the query plan to make sure the index capabilities are being used.
print("\n--- Query Plan ---")

cursor.execute("EXPLAIN QUERY PLAN " + query6)
query_plan = cursor.fetchall()
print(pd.DataFrame(query_plan))



--- Query Plan ---
    0  1  2                                                  3
0   6  0  0                       SCAN TABLE InvoiceLine AS ii
1   8  0  0  SEARCH TABLE Invoice AS i USING INTEGER PRIMAR...
2  11  0  0  SEARCH TABLE Customer AS c USING INTEGER PRIMA...
3  14  0  0  SEARCH TABLE Track AS t USING INTEGER PRIMARY ...


In [ ]:
conn.close()